#### Interval Scheduling

In this problem, we have a set of `requests/jobs` $A=\{1,2,..,n\}$, the $ith$ request corresponds to an interval with `start-time` $s(i)$ and `finish-time` $f(i)$. We have a single `resource` that can fulfill at most one request in a given interval of time. If two or more requests have overlapping intervals, then they are said to be `incompatible`. Our goal then is to `maximize the number of compatible requests that can be fullfilled by the resource`. 

<img src="intervals.png" width="400" height="100">

In this example (borrowed from Kleinberg textbook), we can see that an optimal set of compatible requests that can be fulfilled is given by $R=\{1,3,5,8\}$ (in this case, we clearly have two optimal sets with $|R|=4$).

A simple `greedy` algorithm for finding an optimal solution is the following: 

1) Given the set of requests $A$ and empty set $R$
2) Find the request $i \in A$ which has the earliest finish-time $f(i)$, add $i$ to $R$, then remove $i$ and all requests incompatible with $i$ from $A$
3) If A is not empty, go back to step 2, otherwise return R which is an optimal solution   

Intuitively, since we're always picking the next avaliable request that finishes the earliest, we can get started with the next the request sooner which allows us to fullfill the largest number of requests. (For formal proof by induction, see Kleinberg textbook). 

To implement this algorithm, we can first sort the requests in $A$ by their finish-time and let $i=0$. Then we start by popping out the first request in the sorted list and putting it in $R$. To find all incompatible requests, we simply scan all requests $j$ from the $ith$ position onwards and check whether $s(j) <= f(i)$ which is the condition for incompatibility/overlap, then all requests before the first $j$ for which $s(j) > f(i)$ are incompatibe, so will not be considered further and we start again from $i=j$.

In [9]:
# implementation
def interval_schedule(A):
    # first, sort the intervals by finish time
    A.sort(key=lambda x: x[2])
    # initialize solution list
    R = []
    # find optimal solution
    i = 0
    while i<len(A):
        # add earliest finishing request
        R.append(A[i][0])
        # find next compatible request
        for j in range(i+1, len(A)+1):
            if j < len(A):
                if (A[j][1] >= A[i][2]):
                    break
        i = j

    return R    


In [12]:
# example requests, each request is a tuple (request_id, start_time, finish_time)
A = [(1,0,1), (2,0,1.5), (3,1.4,2.5), (4,2.1,2.6), (5,2.9,4), (6,0,4.2), (7,3.25,4.3), (8,4.8,5.5), (9,4.5,6)]

# find optimal solution
R = interval_schedule(A)
print(f"Optimal set of compatible requests: {R}")

Optimal set of compatible requests: [1, 3, 5, 8]


#### Interval Partitioning

Now we consider a variation of the interval scheduling problem, called the `interval partitioning` problem. We again have a set of `requests/jobs/intervals` $A=\{1,2,..,n\}$, however this time, we have many avaialble resources running in parallel, where each resource can accomodate a single request in a given interval of time. Our goal now is to partition all the requests across different resources, while minimizing the total number of resources used up. 

<img src="partition.png" width="430" height="100">

In this example, we can see that the fewest number of resources across which we can schedule all the requests is `3`. This number simply corresponds to the maximum number of overlapping intervals at any given time, which we define as the `depth`. For this given example set $A$, `depth = 3`. To find the depth for any given set $A$, we can use a slightly modified version of our greedy interval scheduling algorithm:

We first sort the requests in $A$ by their finish-time and initialize $i=0$ and $d=0$. Then we start by popping out the first request in the sorted list and putting it in $R$. To find all incompatible requests, we simply scan all requests $j$ from the $ith$ position onwards and check whether $s(j) <= f(i)$ which is the condition for incompatibility/overlap, then all requests before the first $j$ for which $s(j) > f(i)$ are overlapping, so will not be considered further and we start again from $i=j$. Since all requests from $i$ up to $j-1$ are overlapping, we can keep track of the maximum number of overlapping requests seen so far by simply computing $d = max(j-i, d)$. Then at the end of the algorihm, $d$ is the depth of the interval set $A$.

We need to partitioning the requests across the resources $1,2,..d$. Again, we can use the following simple greedy method. We first order the set by start time (instead of end time), then iterate over each interval $j$ in the ordered list. Then we find each interval $i<j$ which overlaps with $j$ and exclude their labels. We then assign interval $i$ a label from $\{1,2,..,d\}$ which has not been excluded.


In [32]:
from collections import defaultdict

def find_depth(A):
    # first, sort the intervals by finish time
    A.sort(key=lambda x: x[2])
    # initialize solution list
    R = []
    # find optimal solution
    i = 0
    # intialize depth 
    d = 0
    while i<len(A):
        # add earliest finishing request
        R.append(A[i][0])
        # find next compatible request
        for j in range(i+1, len(A)+1):
            if j < len(A):
                if (A[j][1] >= A[i][2]):
                    break
        d = max(d, j-i)
        i = j

    return d   


def interval_partition(A):
    # first, sort the intervals by start time
    A.sort(key=lambda x: x[1])
    # initialize labels with -1 (meaning label not assigned)
    label = {interval[0]:-1 for interval in A}
    # get the depth
    d = find_depth(A)
    # select earliest finishing request
    i = 0
    for j in range(len(A)):
        #print(f"interval j: {A[j][0]}")
        remaining_labels = list(range(d))
        for i in range(0,j):
            # check if interval i overlaps with interval j
            if A[i][2] > A[j][1]:
                # exclude the label of interval i from the list of remaining labels
                remaining_labels.remove(label[A[i][0]])
                #print(f"Found overlapping interval: {A[i][0]}")
        # assign label
        label[A[j][0]] = remaining_labels[0]

    # create a dictionary of requests and their labels
    R = defaultdict(list)
    for request_id, resource_id in label.items():
        R[resource_id].append(request_id)    

    return R   

In [33]:
# example requests
A = [('a',0,1), ('b',0,3), ('c',0,1), ('d',2,3), ('e',2,6), ('f',5,7), ('g',5,7), ('h',6,10), ('i',8,10), ('j',8,10)]


# find optimal solution
R = interval_partition(A)
print(f"Optimal partition of resources: {R}")

Optimal partition of resources: defaultdict(<class 'list'>, {0: ['a', 'd', 'f', 'i'], 2: ['b', 'g', 'j'], 1: ['c', 'e', 'h']})
